In [1]:
import pandas.io.data as web
import pandas as pd
import datetime
import Markowitz
import numpy as np
portfolio = ['YHOO','STK','GOOGL','AMZN','EBAY']
return_markowitz = [0]
return_random = [0]
return_constant = [0]

In [2]:
def pull_returns(symbol):
    request = web.DataReader(symbol,'yahoo',datetime.datetime(2010, 1, 1))['Close']
    returns = request.pct_change()[1:len(request)-1]
    returns *= 100
    return returns

In [3]:
def update_performance(weights,new_returns):
    net_return = 0
    new_returns = list(new_returns)
    for i in range(len(weights)-1):
        net_return = net_return + weights[i]*new_returns[i]
    return net_return
def get_weights_markowitz(return_grid):
    return_grid = np.array(return_grid).transpose()
    weights = Markowitz.optimal_portfolio_quad(return_grid)
    return weights
def get_weights_random(portfolio):
    return np.random.dirichlet(np.ones(len(portfolio)),size=1).tolist()[0]
def get_weights_constant(portfolio):
    return [1/len(portfolio) for x in portfolio]

In [4]:
returns_grid = []
for asset in portfolio:
    returns_grid.append(pull_returns(asset))
returns_grid = pd.DataFrame(returns_grid).transpose()

In [5]:
known_data = returns_grid['2010-01-05':'2015-01-05']
dates = list(returns_grid.index)
today_index = dates.index(pd.Timestamp('2015-01-05'))+1

In [7]:
#rebalancing every 5 days
while today_index < len(dates):
    w_markowitz = get_weights_markowitz(known_data)
    w_random = get_weights_random(portfolio)
    w_constant = get_weights_constant(portfolio)
    i = 0
    while i < 5 and today_index+i < len(dates):
        new_returns = returns_grid.ix[today_index+i]
        return_markowitz.append(return_markowitz[len(return_markowitz)-1]+update_performance(w_markowitz,new_returns))
        return_random.append(return_random[len(return_random)-1]+update_performance(w_random,new_returns))
        return_constant.append(return_constant[len(return_constant)-1]+update_performance(w_constant,new_returns))
        known_data = known_data.append(new_returns)
        know_data = known_data.ix[5:]
        i = i + 1
    today_index = today_index + 5

[0,
 -1.0990522676741721,
 -0.7086616605865691,
 1.433433319496404,
 0.34510842594677116,
 -1.738999562912112,
 -1.7492740957666972,
 -2.6741356493629054,
 -4.2922980244603206,
 -3.0133139428998472,
 -1.9526749285452616,
 -0.95727952359379431,
 1.0443228757812952,
 1.2567280120917415,
 1.0445567529601514,
 -0.89798799637874138,
 -1.893266902824785,
 -2.7904280189325883,
 -0.98131770519976458,
 0.043629255603906181,
 0.35057118391593067,
 -0.14174672865356241,
 0.457818940345468,
 -0.35049929897629528,
 -0.89840951724047091,
 0.3022096225106341,
 0.95776830760666642,
 1.2481605647992424,
 1.9254132085245721,
 0.99236485673593777,
 0.82580451800131172,
 1.904632319390537,
 1.9020038668585482,
 1.4886704644950792,
 1.5219242520582645,
 2.8905358242820745,
 2.4253755085237581,
 1.9008526216714587,
 2.243138473505466,
 1.1200172784321192,
 2.0425887441076842,
 3.3298816149452262,
 1.8562809854051081,
 2.1256157627355341,
 0.87616699411534182,
 0.24949653541817685,
 1.190904249714714,
 0.523

In [13]:
#plot the performance
import matplotlib.pyplot as plt
start_index = dates.index(pd.Timestamp('2015-01-05'))
end_index = len(dates)
trading_dates = [date.to_datetime() for date in dates[start_index:end_index]]
fig= plt.figure()
plt.plot(trading_dates, return_markowitz)
plt.plot(trading_dates, return_random)
plt.plot(trading_dates, return_constant)
plt.ylabel('returns (%)')
plt.legend(['markowitz', 'random', 'constant'], loc='upper left')
plt.title('long only performance of the portfolio YHOO, STK, GOOGL, AMZN, EBAY with different management strategies between March 2015 and October 2016')
plt.show()
fig.savefig("comparative_management.pdf")